In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import joblib

#loading the dataset
df=pd.read_csv("dataset\Cleaned_dataset.csv")

#changing one column name from class to classes
df.rename(columns={'Class': 'Classes'}, inplace=True)

In [30]:
df

,Date_of_journey,Journey_day,Airline,Flight_code,Classes,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Fare
0,2023-01-16,Monday,SpiceJet,SG-8169,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5335
1,2023-01-16,Monday,Indigo,6E-2519,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,5899
2,2023-01-16,Monday,GO FIRST,G8-354,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,5801
3,2023-01-16,Monday,SpiceJet,SG-8709,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5794
4,2023-01-16,Monday,Air India,AI-805,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,2023-03-06,Monday,Vistara,UK-926,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,65028
452084,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452085,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452086,2023-03-06,Monday,Vistara,UK-946,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,72980


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date_of_journey    452088 non-null  object 
 1   Journey_day        452088 non-null  object 
 2   Airline            452088 non-null  object 
 3   Flight_code        452088 non-null  object 
 4   Classes            452088 non-null  object 
 5   Source             452088 non-null  object 
 6   Departure          452088 non-null  object 
 7   Total_stops        452088 non-null  object 
 8   Arrival            452088 non-null  object 
 9   Destination        452088 non-null  object 
 10  Duration_in_hours  452088 non-null  float64
 11  Days_left          452088 non-null  int64  
 12  Fare               452088 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 44.8+ MB


In [32]:
df['Days_left'] = pd.to_numeric(df['Days_left'], downcast='integer')


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date_of_journey    452088 non-null  object 
 1   Journey_day        452088 non-null  object 
 2   Airline            452088 non-null  object 
 3   Flight_code        452088 non-null  object 
 4   Classes            452088 non-null  object 
 5   Source             452088 non-null  object 
 6   Departure          452088 non-null  object 
 7   Total_stops        452088 non-null  object 
 8   Arrival            452088 non-null  object 
 9   Destination        452088 non-null  object 
 10  Duration_in_hours  452088 non-null  float64
 11  Days_left          452088 non-null  int8   
 12  Fare               452088 non-null  int64  
dtypes: float64(1), int64(1), int8(1), object(10)
memory usage: 41.8+ MB


In [34]:
df['Fare'].nlargest()

445107    143019
144044    138951
364805    138496
306424    137275
364804    131283
Name: Fare, dtype: int64

In [35]:
df['Fare'] = pd.to_numeric(df['Fare'], downcast='integer')


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date_of_journey    452088 non-null  object 
 1   Journey_day        452088 non-null  object 
 2   Airline            452088 non-null  object 
 3   Flight_code        452088 non-null  object 
 4   Classes            452088 non-null  object 
 5   Source             452088 non-null  object 
 6   Departure          452088 non-null  object 
 7   Total_stops        452088 non-null  object 
 8   Arrival            452088 non-null  object 
 9   Destination        452088 non-null  object 
 10  Duration_in_hours  452088 non-null  float64
 11  Days_left          452088 non-null  int8   
 12  Fare               452088 non-null  int32  
dtypes: float64(1), int32(1), int8(1), object(10)
memory usage: 40.1+ MB


converted the int64 datatypes to int8 and int16 to reduce the memory size of the model 

In [37]:
df['Date_of_journey'] = pd.to_datetime(df['Date_of_journey'])
#converting the date column to datetime object

# Extract features from "Date_of_journey" column
df['Year'] = df['Date_of_journey'].dt.year
df['Month'] = df['Date_of_journey'].dt.month
df['Day'] = df['Date_of_journey'].dt.day
df['Day_of_week'] = df['Date_of_journey'].dt.dayofweek

# Drop the original "Date_of_journey" column because we dont need it
df.drop('Date_of_journey', axis=1, inplace=True)

Dropped the date of journey column after extracting the year, date and month

dropping the journey day column because we have already extracted it converted into numeric format

In [38]:
df.drop('Journey_day', axis=1, inplace=True)

Dropping the flightcode column because we dont need it . and we can not consider it as an input feature

In [39]:
df.drop('Flight_code', axis=1, inplace=True)

In [40]:
df

,Airline,Classes,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Fare,Year,Month,Day,Day_of_week
0,SpiceJet,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5335,2023,1,16,0
1,Indigo,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,5899,2023,1,16,0
2,GO FIRST,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,5801,2023,1,16,0
3,SpiceJet,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5794,2023,1,16,0
4,Air India,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,5955,2023,1,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,Vistara,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,65028,2023,3,6,0
452084,Vistara,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254,2023,3,6,0
452085,Vistara,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254,2023,3,6,0
452086,Vistara,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,72980,2023,3,6,0


Splitting up X and y . X is a dataframe of features and y is the target variable

In [41]:
X = df.drop(columns=['Fare'],axis=1)
y = df['Fare']

In [42]:
X

,Airline,Classes,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Year,Month,Day,Day_of_week
0,SpiceJet,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,2023,1,16,0
1,Indigo,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,2023,1,16,0
2,GO FIRST,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,2023,1,16,0
3,SpiceJet,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,2023,1,16,0
4,Air India,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,2023,1,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,Vistara,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,2023,3,6,0
452084,Vistara,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,2023,3,6,0
452085,Vistara,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,2023,3,6,0
452086,Vistara,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,2023,3,6,0


In [43]:
y

0          5335
1          5899
2          5801
3          5794
4          5955
          ...  
452083    65028
452084    69254
452085    69254
452086    72980
452087    72980
Name: Fare, Length: 452088, dtype: int32

In [44]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Airline            452088 non-null  object 
 1   Classes            452088 non-null  object 
 2   Source             452088 non-null  object 
 3   Departure          452088 non-null  object 
 4   Total_stops        452088 non-null  object 
 5   Arrival            452088 non-null  object 
 6   Destination        452088 non-null  object 
 7   Duration_in_hours  452088 non-null  float64
 8   Days_left          452088 non-null  int8   
 9   Year               452088 non-null  int64  
 10  Month              452088 non-null  int64  
 11  Day                452088 non-null  int64  
 12  Day_of_week        452088 non-null  int64  
dtypes: float64(1), int64(4), int8(1), object(7)
memory usage: 41.8+ MB


Converting year, month, day, dayofweek columns from int64 to int16

In [45]:
X['Year'] = pd.to_numeric(X['Year'], downcast='integer')
X['Month'] = pd.to_numeric(X['Month'], downcast='integer')
X['Day'] = pd.to_numeric(X['Day'], downcast='integer')
X['Day_of_week'] = pd.to_numeric(X['Day_of_week'], downcast='integer')

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Airline            452088 non-null  object 
 1   Classes            452088 non-null  object 
 2   Source             452088 non-null  object 
 3   Departure          452088 non-null  object 
 4   Total_stops        452088 non-null  object 
 5   Arrival            452088 non-null  object 
 6   Destination        452088 non-null  object 
 7   Duration_in_hours  452088 non-null  float64
 8   Days_left          452088 non-null  int8   
 9   Year               452088 non-null  int16  
 10  Month              452088 non-null  int8   
 11  Day                452088 non-null  int8   
 12  Day_of_week        452088 non-null  int8   
dtypes: float64(1), int16(1), int8(4), object(7)
memory usage: 30.2+ MB


Train test splitting the datas using sklearn

In [47]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# creating the pipeline for data transformation and training 

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

In [49]:
#dividing numerical and categorical column
numeric_columns=['Duration_in_hours','Day_of_week','Day','Month','Year','Days_left']
cat_columns= ['Airline', 'Classes', 'Source','Departure','Total_stops', 'Arrival', 'Destination']

In [50]:
numeric_transformer = StandardScaler()
#using the standard scaler for numeric datas
categorical_transformer = OneHotEncoder()
#using onehot encoder for categorical datas

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, cat_columns)
    ])


In [51]:
model = RandomForestRegressor()
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])